In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('covtype.data', header=None)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [7]:
df[54].unique()

array([5, 2, 1, 7, 3, 6, 4])

In [8]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

Using TensorFlow backend.


In [15]:
class CrossLayer(Layer):
    def __init__(self, layer_num=1, bais=False, res_connect=False, **kwargs):
        self.layer_num = layer_num
        self.bais = bais
        self.res_connect = res_connect
        super(CrossLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.ws = []
        if self.bais:
            self.bs = []
        for i in range(self.layer_num):
            self.ws.append(self.add_weight(name='cross_w_%d' % i,
                                          shape=(input_shape[1],1),
                                          initializer='uniform',
                                          trainable=True))
            if self.bais:
                self.bs.append(self.add_weight(name='cross_b_%d' % i,
                                          shape=(input_shape[1],1),
                                          initializer='uniform',
                                          trainable=True))
        super(CrossLayer, self).build(input_shape)
    
    def call(self, x):
        xs = [x]
        for i in range(self.layer_num):
            out = K.dot(xs[i], self.ws[i]) * x
            if self.bais:
                out += self.bs[i]
            if self.res_connect:
                out += xs[i]
            xs.append(out)
        return xs[-1]
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [47]:
from keras.layers import InputLayer, Dense
from keras.models import Sequential

In [53]:
model = Sequential([
        CrossLayer(layer_num=4, input_shape=(32,) ),
        Dense(1)
    ])

In [54]:
x = np.random.randn(100000, 32)
y = np.sum(np.exp(0.1*x), axis=1)

In [57]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [51]:
import keras
tfcb = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [58]:
model.fit(x, y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[tfcb])

Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 3s - loss: 0.2588 - mean_squared_error: 0.2588 - val_loss: 0.2639 - val_mean_squared_error: 0.2639
Epoch 2/50
80000/80000 [==============================] - 3s - loss: 0.2595 - mean_squared_error: 0.2595 - val_loss: 0.2638 - val_mean_squared_error: 0.2638
Epoch 3/50
80000/80000 [==============================] - 3s - loss: 0.2593 - mean_squared_error: 0.2593 - val_loss: 0.2654 - val_mean_squared_error: 0.2654
Epoch 4/50
80000/80000 [==============================] - 3s - loss: 0.2562 - mean_squared_error: 0.2562 - val_loss: 0.2591 - val_mean_squared_error: 0.2591
Epoch 5/50
80000/80000 [==============================] - 3s - loss: 0.2612 - mean_squared_error: 0.2612 - val_loss: 0.2645 - val_mean_squared_error: 0.2645
Epoch 6/50
80000/80000 [==============================] - 3s - loss: 0.2572 - mean_squared_error: 0.2572 - val_loss: 0.2528 - val_mean_squared_error: 0.2528
Epoch 7/

In [ ]:
df = pd.read_csv('')